## Generic setup for the CW Nano

In [2]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
%run "../../../Helper_Scripts/Setup_Generic.ipynb"
scope._cwusb.usbtx.rep = 0x81
scope._cwusb.usbtx.wep = 0x02

INFO: Found ChipWhisperer😍


## Calling the path for the hex file and programming the board

In [3]:
fw_path = "../recall-CWNANO.hex"
cw.program_target(scope, cw.programmers.STM32FProgrammer, fw_path)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 4983 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 4983 bytes


######################################################

## Attack strategies
To perform the attack we tried to find the vulnerability in the C code first. We found that for the verify function it tries to match with the memory one at a time. As soon as we match one it goes for the  next index. As the code has the "break" in line 18, it breaks running the code if not matched but it keep on runing if it matches. This tells us that as soon as the memory is matched it has longer execution time. 

As the memory is 16 byte and each byte has the possibility of 0 to 255. Therefore we made an arry of [0]*16. Then in a loop we change the first index from 0 to 255 to find the first index memory. If the input matches then it goes for next iteration. 

We used the correlation function to match with the power traces with each other to find when more time is needed. As soon as we see that the correlation dropped that means that it takes more time and the verify function goes to the next inderx to verify. We can take the index when the correlation drops. That will be the memory element. Then we update the memory array and for the next memory index to find that.

We can totally have 256*16 combination to find the correct memory element. 
However our code is written in efficient way to find the memory faster. As soon as it get a memory element it goes to find the next one.


In [4]:
import numpy as np
import time
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


msg = bytearray([0]*16)
for k in range (0, 16):
    c_trace=np.zeros([3,300], dtype=float)
    
    #0
    msg[k]=0
    scope.arm()
    target.simpleserial_write('a',msg)
    scope.capture()
    c_trace[0,:]=scope.get_last_trace()[0:300]
    
    #1
    msg[k]=1
    scope.arm()
    target.simpleserial_write('a',msg)
    scope.capture()
    c_trace[1,:]=scope.get_last_trace()[0:300]
    
    #2
    msg[k]=2
    scope.arm()
    target.simpleserial_write('a',msg)
    scope.capture()
    c_trace[2,:]=scope.get_last_trace()[0:300]
    
    corr_0_1, _ = pearsonr(c_trace[0,:], c_trace[1,:])
    corr_0_2, _ = pearsonr(c_trace[0,:], c_trace[2,:])
    corr_1_2, _ = pearsonr(c_trace[1,:], c_trace[2,:])
    
    threshold=0.8 # a threshold is is used to find the correlation drop.
    
    if(corr_0_1 < threshold* corr_0_2):
        msg[k]=1
    elif(corr_0_1 < threshold*corr_1_2 ):
        msg[k]=0
    else:
        for i in range (2,256):
            msg[k] = i

            
            scope.arm()
            target.simpleserial_write('a',msg)
            scope.capture()
            signal=scope.get_last_trace()
            
            corr_0_i, _ = pearsonr(c_trace[0,:], signal[0:300])
            
            if(corr_0_i < threshold* corr_0_1):
                ##uncomment if you want to plot#
                '''plt.figure(figsize=(18.5, 6.5), dpi=150)
                axes = plt.gca()
                plt.plot(c_trace[0,:].T, linewidth=2, label ="Memory Not Matched" )
                plt.plot(signal[0:300].T, linewidth=2, label =" Memory Matched" )
                leg = axes.legend(loc ="upper left")
                plt.title("Index:%d, Found Memory: %s" %(k, hex(msg[k])))'''
                
                break
    
    print("Memory:", msg )          
    

Memory: CWbytearray(b'70 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 00 00 00 00 00 00 00 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 00 00 00 00 00 00 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 00 00 00 00 00 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 00 00 00 00 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 00 00 00 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 73 00 00 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 73 6b 00 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 73 6b 6e 00 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 73 6b 6e 30 00 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 73 6b 6e 30 77 00 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 73 6b 6e 30 77 6c 00 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 73 6b 6e 30 77 6c 33 00 00 00')
Memory: CWbytearray(b'70 30 77 33 52 31 73 6b 6e 30 77 6c 33 64 

## Verify

Here we are verifying the found memory and Goal shoul be '01'.
Note that, we found the memory as:

CWbytearray(b'70 30 77 33 52 31 73 6b 6e 30 77 6c 33 64 67 33')

bytearray([112,48,119,51,82,49,115,107,110,48,119,108,51,100,103,51])

In [5]:
#CWbytearray(b'70 30 77 33 52 31 73 6b 6e 30 77 6c 33 64 67 33')
#msg = bytearray([112,48,119,51,82,49,115,107,110,48,119,108,51,100,103,51])
print("Found Memory:",msg)
target.simpleserial_write('a',msg)
Goal=target.simpleserial_read('r', 1)
print("Goal:",Goal)

Found Memory: CWbytearray(b'70 30 77 33 52 31 73 6b 6e 30 77 6c 33 64 67 33')
Goal: CWbytearray(b'01')
